## Семинар 1 Индекс

## Intro

### чтение файла 
- конструкция __with open__ (recommended)
- конструкция __open + close__

In [211]:
fpath = 'fpath.txt'

# одним массивом  
with open(fpath, 'r') as f:  
    text = f.read() 

#по строкам, в конце каждой строки \n  
with open(fpath, 'r') as f:   
    text = f.readlines() 

#по строкам, без \n   
with open(fpath, 'r') as f:   
    text = f.read().splitlines() 
    
#not reccomended  
file = open(txt_fpath, 'r')  
text = file.read()    
file.close() 

### работа с файлами и папками

### os.path  
путь до файла

In [ ]:
import os

# возвращает полный путь до папки/файла по имени файла / папки
print(os.path.abspath('fpath.txt'))

# возвращает имя файла / папки по полному пути до него
print(os.path.basename('/your/path/to/folder/with/fpath.txt'))

# проверить существование директории - True / False
print(os.path.exists('your/path/to/any/folder/'))

### os.listdir  
возвращает список файлов в данной директории

In [ ]:
main_dir = '/your/path/to/folder/with/folders/'
os.listdir(main_dir)

сделаем пути абсолютными, чтобы наш код не зависел от того, где лежит этот файл

In [ ]:
[main_dir + fpath for fpath in os.listdir(main_dir)]

не забывайте исключать системные директории, такие как .DS_Store

In [ ]:
[main_dir + fpath for fpath in os.listdir(main_dir) if not '.DS_Store' in fpath]

### os.walk
root - начальная директория  
dirs - список поддиректорий (папок)   
files - список файлов в этих поддиректориях  

In [ ]:
main_dir = '/your/path/to/folder/with/folders/'

for root, dirs, files in os.walk(main_dir):
    for name in files:
        print(os.path.join(root, name))

> __os.walk__ возвращает генератор, это значит, что получить его элементы можно только проитерировавшись по нему  
но его легко можно превратить в list и увидеть все его значения

In [ ]:
list(os.walk(main_dir))

##  Обратный индекс 

Сам по себе обратный индекс не может осуществлять поиск, для этого необходимо добавить к нему определенную метрику. Это не совсем очевидная задача, поэтому немного отложим ее. А сейчас посмотрим, что полезного можно вытащить из индекса.    
По сути, индекс - это информация о частоте встречаемости слова в каждом документе.   
Из этого можно понять, например:
1. какое слово является самым часто употребимым / редким
2. какие слова встречаются всегда вместе. Так можно парсить твиттер, fb, форумы и отлавливать новые устойчивые выражения в речи
3. какой документ является самым большим / маленьким (очень изощренный способ, когда есть _len_)

In [98]:
import pandas as pd
import numpy as np
import os
import string
from nltk.corpus import stopwords
from string import punctuation
from collections import defaultdict, Counter
import warnings
from pymystem3 import Mystem

mystem = Mystem()
warnings.filterwarnings('ignore')

### __Задача__: 
получите обратный индекс для коллекция документов.    
Перед этим постройте матрицу терм-документ и сделайте функцию булева поиска, которая по запросу будет возвращать 5 релевантных документов.   
В качестве коллекции возьмите сценарий сезонов сериала Друзья. Одна серия - один документ.

[download_friends_corpus](https://yadi.sk/d/k_M7n63A3adGSz)

Этапы:   
    1. получить коллекцию документов   --- +
    2. для каждого файла коллекции сделать необходимую на ваш взгляд предобработку --- +
    3. получить матрицу терм-документ, написать функцию поиска по ней --- +
    4. получить обратный индекс в виде словаря, где ключ - нормализованное слово,
    значение - список файлов, в которых это слово встречается  --- +
    5. вывести кусочек индекса в виде таблицы --- +
    6. сделать анализ обратного индекса. Это задание принимается в виде кода и ответов на вопросы --- +

In [3]:
main_dir = 'Friends/'

In [6]:
stops = stopwords.words('russian')

In [8]:
import re

def remove(text):
    new = re.sub(r'\ufeff', '', text)
    new = re.sub(r'\n', ' ', new)
    new = re.sub(r'www.tvsubtitles.net', '', new)
    new = re.sub(r'([\W]|\d)', ' ', new)
    new = re.sub('(\n|\t|\v|\r)', ' ', new)
    return new

In [13]:
def data_info_creator(main_dir):

    all_ = defaultdict(str)  # episod - text
    idx = defaultdict(str)  # idx - episod
    seasons = defaultdict(list)  # season - [episods]
    
    avr = 0
    i = 0
    l = 0

    for root, dirs, files in os.walk(main_dir):
        for name in files:
            if not '.DS_Store' in name:
                with open(os.path.join(root, name), 'r') as f:
                    text = f.read() 
                    a = mystem.lemmatize(remove(text))
                    avr += len(a)
                    all_[name[:-7]] = ''.join(a)
                    idx[i] = name[:-7]
                    seasons[l].append(name[:-7])
                    i += 1
        l += 1
    return all_, idx, seasons, avr / i

In [127]:
def data_info_creator(main_dir):

    all_data = defaultdict(dict)  # название файла: {text, len}
    word_count = defaultdict(dict)  # word : {id: count}
    idx = defaultdict(str)  # idx - episod
    seasons = defaultdict(list)  # season - [episods]

    avr = 0
    i = 0
    l = 0
    global mystem

    for root, dirs, files in os.walk(main_dir):
        for name in files:
            if not '.DS_Store' in name:
                with open(os.path.join(root, name), 'r', encoding='utf-8') as f:
                    text = f.read() 
                    words = mystem.lemmatize(remove(text))
                    length = len(words)
                    avr += length
                    all_data[name[:-7]] = {'text': ''.join(words),
                                           'len': length}
                    seasons[l].append(name[:-7])
                    idx[i] = name[:-7]
                    prob = Counter(words)
                    for word in prob:
                        word_count[word][name[:-7]] = prob[word]
                    i += 1
        l += 1
    return all_data, word_count, idx, seasons, avr 

In [128]:
all_data, word_count, idx, seasons, avrdl = data_info_creator(main_dir)

In [116]:
f = open('/Users/Stoneberry/Desktop/Uni/Прога/4 курс/Поиск/лекции/Friends/Friends - season 7/Friends - 7x09 - The One With All The Candy.ru.txt', 'r', encoding='utf-8')
arr = f.read()
f.close()
words = mystem.lemmatize(remove(arr))


Пример текста

In [102]:
print(all_data['Friends - 1x01 - The One Where Monica Gets A Roommate']['text'][0:200])

друг  как все начинаться  да нечего рассказывать  он просто сотрудник   ладно ты  ты же на свидание с он собираться   значит  он не мочь не быть с придурь   джой  вести себя прилично  так у он горб  и


In [104]:
print(all_data['Friends - 1x01 - The One Where Monica Gets A Roommate']['text'][-200:])

  мисс  можно кофе   извинять  не мочь бы вы относить это туда   давать  давать  спасибо  извинять  так так  ласа вегас     итак  я в ласа вегас  и я   лайза минелли                                  



In [105]:
len(all_data)

165

Функция поиска

In [137]:
def data_search(word_count, word, episode=None, non=False):

    if not isinstance(word, str):
        raise ValueError
    if len(word.split(' ')) > 1:
        raise ValueError
    if word == '':
        return None

    word = word.lower()
    answer = word_count[word]
    if answer != {}:
        if episode is None:
            return set(answer.keys())
        return answer[episode]
    return ValueError('There is no such word')

In [138]:
data_search(word_count, 'лекарство')

{'Friends - 3x17 - The One Without The Ski Trip',
 'Friends - 4x02 - The One With The Cat.Tv',
 "Friends - 6x03 - The One With Ross's Denial",
 'Friends - 6x07 - The One Where Phoebe Runs',
 'Friends - 6x11 - The One With The Apothecary Table'}

In [139]:
data_search(word_count, 'в', 'Friends - 4x01 - The One With The Jellyfish.Tv')

20

Обратный индекс

![](Friends/wedding.png)

Напоминание:    
> При итерации по списку вы можете помимо самого элемента получить его порядковый номер    
``` for i, element in enumerate(your_list): ...  ```    
Иногда для получения элемента делают так -  ``` your_list[i] ```, старайтесь этого избегать

In [26]:
main_dir = '/your/path/with/Frinds/collection/'
files_list = []

### пройдитесь по всем папкам коллекции и соберите все пути .txt файлов
### _check : в коллекции должно быть 165 файлов


In [55]:
def space(text):
    new = re.sub('\(', '( ', text)
    new = re.sub('\)', ' )', new)
    return new

In [57]:
### постройте матрицу терм-документ
term_doc_matrix = []


### напишите функцию булева поиска по построенной матрице
def boolean_search(data, text):
    """
    Produces a Boolean search according with the term-document matrix
    :return: list of first 5 relevant documents
    """
    words = []
    op = []
    oper = ['&', 'ИЛИ', 'НЕ', '(', ')']
    text = space(text)
    text = text.split(' ')

    for item in text:

        if item not in oper:
            if len(op) > 0 and op[-1] == 'НЕ':
                item = data_search(data, item.lower(), non=True)
                op.pop()
            else:
                item = data_search(data, item.lower())

            if len(words) == 0:
                words.append(item)

            elif len(op) > 0:
                if op[-1] == '&':
                    answ = words[-1] & item
                    words.pop()
                    words.append(answ)
                    op.pop()
                elif op[-1] == 'ИЛИ':
                    answ = words[-1] | item
                    words.pop()
                    words.append(answ)
                    op.pop()
                elif op[-1] == '(':
                    op.pop()
                    words.append(item)

        elif item == ')':

            if len(op) <= 0:
                raise ValueError('Enter correct string')

            if op[-1] == '(':
                if len(words) <= 0:
                    raise ValueError('Enter correct string')

            else:
                for i in op[::-1]:

                    if i == '(':
                        break
                    elif i == '&':
                        answ = words[-2] & words[-1]
                        for i in range(2):
                            words.pop()
                        words.append(answ)
                        op.pop()
                    elif i == 'ИЛИ':
                        answ = words[-2] | words[-1]
                        for i in range(2):
                            words.pop()
                        words.append(answ)
                        op.pop()
                if len(op) == 0:
                    break

        else:
            op.append(item)
    return words[-1]


#запросы 
input_text = [
    'Моника & Фиби & Рэйчел & Чендлер & Джоуи & Росс',
    '(Моника ИЛИ Фиби) & Рэйчел & (Чендлер ИЛИ Джоуи) & Росс',
    '(НЕ Моника) & Фиби & Рэйчел & Чендлер & Джоуи & (НЕ Росс)'
]

In [130]:
text = 'Моника & Фиби & Рэйчел & Чендлер & Джоуи & Росс'
boolean_search(word_count, text)

{'Friends - 1x07 - The One With The Blackout',
 'Friends - 1x11 - The One With Mrs. Bing',
 'Friends - 1x14 - The One With The Candy Hearts',
 'Friends - 1x15 - The One With The Stoned Guy',
 'Friends - 2x06 - The One With The Baby On The Bus',
 'Friends - 3x06 - The One With The Flashback',
 'Friends - 4x01 - The One With The Jellyfish.Tv',
 "Friends - 5x22 - The One With Joey's Big Break",
 'Friends - 6x01 - The One After Vegas',
 "Friends - 6x03 - The One With Ross's Denial",
 'Friends - 6x06 - The One On The Last Night',
 'Friends - 6x07 - The One Where Phoebe Runs',
 'Friends - 6x09 - The One Where Ross Got High',
 "Friends - 6x21 - The One Where Ross Meets Elizabeth's Dad",
 "Friends - 6x22 - The One Where Paul's The Man",
 'Friends - 6x23 - The One With The Ring',
 "Friends - 7x01 - The One With Monica's Thunder",
 "Friends - 7x03 - The One With Phoebe's Cookies",
 "Friends - 7x04 - The One With Rachel's Assistant",
 'Friends - 7x05 - The One With The Engagement Picture',
 'Frie

<img src="pics/inv_index3.svg" alt="Drawing" style="width: 600px;"/>

Совет для построения обратного индекса: 
> В качестве словаря используйте ``` defaultdict ``` из модуля collections   
Так можно избежать конструкции ``` dict.setdefault(key, default=None) ```

In [91]:
def inverted_index(data) -> dict:
    """
    Create inverted index by input doc collection
    :return: inverted index
    """

    d = defaultdict(list)

    for word in data.keys():
        d[word] = [i for i in data_search(data, word)]

    return d

In [93]:
d = inverted_index(data)

In [94]:
d['and']

['Friends - 4x01 - The One With The Jellyfish.Tv',
 "Friends - 6x19 - The One With Joey's Fridge"]

#### Теперь для этого есть словарь word_count

С помощью обратного индекса произведите следующую аналитику:  

1) общая аналитика
- какое слово является самым частотным?
- какое самым редким?
- какой набор слов есть во всех документах коллекции?

2) частота встречаемости имен главных героев в каждом сезоне      
- какой сезон был самым популярным у Чендлера? у Моники?   
- кто из главных героев статистически самый популярный? 


Таблица индекса

In [275]:
inv_ind = pd.DataFrame({'term': vect.get_feature_names(),
                        'docs': list(d.values()),
                        'freq': data['freq']
                       })

In [276]:
inv_ind['lenght'] = inv_ind.docs.map(lambda x: len(x))

In [277]:
inv_ind.head()

,docs,freq,term,lenght
Term,,,,
after,[Friends - 7x22 - The One With Chandler's Dad],0.025744,after,1
again,[Friends - 7x12 - The One Where They're Up All...,0.025580,again,1
ahh,[Friends - 4x04 - The One With The Ballroom Da...,0.024608,ahh,35
all,[Friends - 7x15 - The One With Joey's New Brai...,0.068349,all,3
and,[Friends - 6x16-17 - The One That Could Have B...,0.076152,and,17


Какое слово является самым частотным?

In [278]:
inv_ind.ix[inv_ind['freq'].idxmax()]

docs      [Friends - 2x04 - The One With Phoebe's Husban...
freq                                                 55.332
term                                                     ты
lenght                                                    2
Name: ты, dtype: object

Какое самым редким?

In [279]:
inv_ind.ix[inv_ind['freq'].idxmin()]

docs      [Friends - 1x16 - The One With Two Parts (1)]
freq                                          0.0156364
term                                                 би
lenght                                                1
Name: би, dtype: object

Какой набор слов есть во всех документах коллекции?

In [280]:
list(inv_ind.loc[inv_ind['lenght'] == 165].term)

['done',
 'oo',
 'агрегат',
 'базар',
 'бублик',
 'забавный',
 'каво',
 'клешня',
 'кой',
 'кофейня',
 'мастерица',
 'мега',
 'молочко',
 'мужесвеннен',
 'насчет',
 'ноль',
 'обставлять',
 'отвратный',
 'переносить',
 'повседневный',
 'повыдергать',
 'привязанность',
 'пчела',
 'расположение',
 'соседский',
 'съезжать',
 'твердо',
 'фаш',
 'фильмец',
 'фрэнсис',
 'халка',
 'червь']

Какой сезон был самым популярным у Чендлера? у Моники?

Чендлер

In [281]:
def popular(name):

    row = data.loc[name.lower()]
    row = row.drop('freq')

    maximum = 0
    seas = ''

    for i in range(1, 8):
        a = row[seasons[i]].sum()
        if maximum < a:
            maximum = a
            seas = i

    return seas

In [282]:
popular('чендлер')

2

Моника

In [283]:
popular('моника')

2

Кто из главных героев статистически самый популярный?

In [284]:
data.loc[['моника', 'росс', 'джоуи', 'чендлер', 'фиби', 'рэйчел']]['freq']

Term
моника     3.881715
росс       5.182460
джоуи      3.999995
чендлер    4.085088
фиби       3.236180
рэйчел     2.427932
Name: freq, dtype: float64

Росс самый популярный

## Функция ранжирования Okapi BM25

Для обратного индекса есть общепринятая формула для ранжирования *Okapi best match 25* ([Okapi BM25](https://ru.wikipedia.org/wiki/Okapi_BM25)).    
Пусть дан запрос $Q$, содержащий слова  $q_1, ... , q_n$, тогда функция BM25 даёт следующую оценку релевантности документа $D$ запросу $Q$:

$$ score(D, Q) = \sum_{i}^{n} \text{IDF}(q_i)*\frac{(k_1+1)*f(q_i,D)}{f(q_i,D)+k_1(1-b+b\frac{|D|}{avgdl})} $$ 
где   
>$f(q_i,D)$ - частота слова $q_i$ в документе $D$ (TF)       
$|D|$ - длина документа (количество слов в нём)   
*avgdl* — средняя длина документа в коллекции    
$k_1$ и $b$ — свободные коэффициенты, обычно их выбирают как $k_1$=2.0 и $b$=0.75   
$$$$
$\text{IDF}(q_i)$ есть обратная документная частота (IDF) слова $q_i$: 
$$\text{IDF}(q_i) = \log\frac{N-n(q_i)+0.5}{n(q_i)+0.5},$$
>> где $N$ - общее количество документов в коллекции   
$n(q_i)$ — количество документов, содержащих $q_i$

Таблица с кол-вом употреблений

In [300]:
new_vect = CountVectorizer()
X = new_vect.fit_transform(list(all_.values())) 
new_data = pd.DataFrame(X.todense()).transpose()
new_data.columns = list(idx.values())
new_data['Term'] = new_vect.get_feature_names()
new_data = new_data.set_index(['Term'])

In [149]:
### реализуйте эту функцию ранжирования
from math import log


def score_BM25(qf, dl, avgdl, k1, b, N, n):
    """
    Compute similarity score between search query and documents from collection
    :return: score

    qf - кол - во вхождений слова в документе
    dl - длина документа

    """
    tf = qf / dl
    idf = log(N - n + 0.5 / n + 0.5)
    a = (k1 + 1) * tf
    b = tf + k1*(1 - b + b*(dl / avgdl))

    return (a / b) * idf

In [311]:
score_BM25(45, 566, avgdl, k1, b, N, 3)

2.0942846416971577

### __Задача__:    
напишите функцию, которая сортирует поисковую выдачу для любого входящего запроса согласно метрике *Okapi BM25*.    
Выведите 10 первых результатов и их скор по запросу **рождественские каникулы**. 

In [146]:
from nltk import word_tokenize

In [142]:
def preprocessing(input_text, stopwords={}, del_stopwords=True, del_digit=True):

    words = [x.lower().strip(string.punctuation+'»«–…') for x in word_tokenize(input_text)]
    lemmas = [mystem.lemmatize(x)[0] for x in words if x]
    lemmas_arr = []
    for lemma in lemmas:
        if del_stopwords:
            if lemma in stopwords:
                continue
        if del_digit:
            if lemma.isdigit():
                continue
        lemmas_arr.append(lemma)
    return lemmas_arr

In [147]:
def compute_sim(words, doc, info_data, word_count, N, avgdl):
    """
    Compute similarity score between search query and documents from collection
    :return: score
    """

    k1 = 2.0
    b = 0.75
    ans = 0

    for word in words:
        if word_count[word] != {}:

            try: qf = word_count[word][doc]
            except KeyError: qf = 0

            dl = info_data[doc]['len']
            n = len(word_count[word])
            ans += score_BM25(qf, dl, avgdl, k1, b, N, n)

    return ans


def get_search_result(text, info_data, word_count, avgdl, stopwords={}, del_stop=True, amount=10):
    """
    Compute sim score between search query and all documents in collection
    Collect as pair (doc_id, score)
    :param query: input text
    :return: list of lists with (doc_id, score)
    """

    if not isinstance(text, str):
        raise ValueError
    
    words = preprocessing(text, stopwords=stopwords, del_stopwords=del_stop, del_digit=True)
    answer = {}
    N = len(info_data)
   
    for doc in info_data:
        answer[doc] = compute_sim(words, doc, info_data, word_count, N, avgdl)

    for index, ans in enumerate(sorted(answer.items(), reverse=True, key=lambda x: x[1])):
        if index >= amount: break
        yield (ans[0], info_data[ans[0]], ans[1])

In [151]:
for i in get_search_result('рождественские каникулы', all_data, word_count, avgdl, stopwords=stops, del_stop=True, amount=10):
    print(i[0], i[2])

Friends - 7x10 - The One With The Holiday Armadillo 0.016545092871755835
Friends - 6x19 - The One With Joey's Fridge 0.00999778323930336
Friends - 3x10 - The One Where Rachel Quits 0.008855711671988632
Friends - 2x09 - The One With Phoebe's Dad 0.005792518010119177
Friends - 1x17 - The One With Two Parts (2) 0.004173275047876691
Friends - 4x03 - The One With The 'Cuffs 0.004018665263927699
Friends - 1x16 - The One With Two Parts (1) 0.003907436398525218
Friends - 4x10 - The One With The Girl From Poughkeepsie 0.0035755676236525857
Friends - 6x10 - The One With The Routine 0.0020095993237095795
Friends - 4x08 - The One With Chandler In A Box 0.0017636645582629223
